<a href="https://colab.research.google.com/github/deepak-ucfknight/Machine_Learning_Spring19/blob/master/Logisitic_Regression_Problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Machine Learning Assignment  - 1 

Deepak - 4736979

In [0]:
# importing datasets from keras

import keras
from keras.datasets import mnist
from keras import backend as K
import numpy as np

Setting Variables

In [0]:
batch_size = 32
num_classes = 1
epochs = 12
classifier_digit = 5

Image Dimensions

In [0]:
rows = 28
cols = 28

Loading MNIST Datasets

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Reshaping the Data  based on the backend format,  i.e. channels first or channels last

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, rows, cols)
    x_test = x_test.reshape(x_test.shape[0], 1, rows, cols)
    input_shape = (1, rows, cols)
else:
    x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
    x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)
    input_shape = (rows, cols, 1)
    

x_train = x_train / 255;
x_test = x_test / 255;


In [0]:
def modify_labels(y):
  y = np.array(y)
  return np.where(y == classifier_digit, 1, 0)
  

y_train = modify_labels(y_train)
y_test = modify_labels(y_test)


In [0]:

x_train = x_train.reshape(x_train.shape[0], -1).T
x_test = x_test.reshape(x_test.shape[0], -1).T
y_train = y_train.T
y_test = y_test.T



Activation Function : Logistic Regression

In [0]:
def sigmoid(z):
   return 1.0/(1.0+np.exp(-z))
  
def sigmoid_prime(z):
  return sigmoid(z)*(1-sigmoid(z))

Helper Functions

In [0]:
# This function initializes the weights matrices and bias to zero

def initialize_with_zeros(dim):
   w = np.zeros(shape=(dim, num_classes))
   b = 0
   return w,b


In [0]:
def mini_batches(X, Y, batchsize):
    for start_idx in range(0, X.shape[0] - batchsize + 1, batchsize):
        excerpt = slice(start_idx, start_idx + batchsize)
        yield X[excerpt], Y[excerpt]

In [0]:
def binary_entropy(w,b,X,Y):
  
  m =  m = X.shape[1]
  A = sigmoid(np.dot(w.T, X) + b)
  cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A))) # binary entropy
 
  # binary entropy gradients
  dw = (1 / m) * np.dot(X, (A - Y).T)
  db = (1 / m) * np.sum(A - Y)
  
  grads = {"dw": dw,
            "db": db }
  
  cost = np.squeeze(cost)
    
  return grads, cost

In [0]:
# gradient calculation has to be updated - under construction
def squared_loss(w,b,X,Y):
  
  m =  m = X.shape[1]
  A = sigmoid(np.dot(w.T, X) + b)
  cost = np.square(Y - A).mean()
 
  # binary entropy gradients
  dw = (1 / m) * np.dot(X, (A - Y).T)
  db = (1 / m) * np.sum(A - Y)
  
  grads = {"dw": dw,
            "db": db }
    
  return grads, cost

Back Propogation of Gradient Descent

In [0]:
def optimize(w, b, X, Y, epochs, learning_rate, print_cost = False):
  
  costs = []
    
  print(X.shape)
  print(Y.shape)
  for i in range(epochs):
      
    for batch in mini_batches(X.T, Y.T, batch_size):
    
       x_batch, y_batch = batch
       grads, cost = binary_entropy(w, b, x_batch.T, y_batch.T)
       dw = grads["dw"]
       db = grads["db"]
       w = w - learning_rate * dw 
       b = b - learning_rate * db

       
    costs.append(cost)

    if print_cost:
       print ("Loss after iteration %i: %f" % (i, cost))
            
  params = {"w": w,
            "b": b}

  grads = {"dw": dw,
               "db": db}

  return params, grads, costs
        

Predicting Values

In [0]:
def predict(w, b, X):
   m = X.shape[1]
   Y_prediction = np.zeros((1, m))
   w = w.reshape(X.shape[0], num_classes)
   A = sigmoid(np.dot(w.T, X) + b)
  
   for i in range(A.shape[1]):
      Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
   return Y_prediction
      

Model

In [0]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
   w, b = initialize_with_zeros(X_train.shape[0])
   parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
   w = parameters["w"]
   b = parameters["b"]
   Y_prediction_test = predict(w, b, X_test)
   Y_prediction_train = predict(w, b, X_train)
  
   print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
   print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
   d = { "costs": costs,
          "Y_prediction_test": Y_prediction_test, 
          "Y_prediction_train" : Y_prediction_train, 
          "w" : w, 
          "b" : b,
          "learning_rate" : learning_rate,
          "num_iterations": num_iterations }
    
   return d

In [135]:
classifier = model(x_train, y_train, x_test, y_test, num_iterations = 12, learning_rate = 0.001, print_cost = True)

(784, 60000)
(60000,)
Loss after iteration 0: 0.489770
Loss after iteration 1: 0.404124
Loss after iteration 2: 0.363584
Loss after iteration 3: 0.342393
Loss after iteration 4: 0.330479
Loss after iteration 5: 0.323417
Loss after iteration 6: 0.319067
Loss after iteration 7: 0.316312
Loss after iteration 8: 0.314530
Loss after iteration 9: 0.313360
Loss after iteration 10: 0.312584
Loss after iteration 11: 0.312067
train accuracy: 90.965 %
test accuracy: 91.08 %
